In [349]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from keras import backend as K
from scipy.spatial import distance

def ComputeMaxDistance(X1, X2):
    max_dist = 0
    for x1 in X1:
        for x2 in X2:
            dist = distance.euclidean(x1, x2)
            max_dist = max(dist, max_dist)
    return max_dist

def RBF_kernel(x, centers, sigma):
  # res = []
  # for c in centers:
    # res.append(math.exp((-np.linalg.norm(x - c)**2) / (2*sigma**2)))
  # res = [math.exp(-np.linalg.norm(x - c)**2) / (2 * sigma**2) for c in centers]
  res = [math.exp(-np.linalg.norm(x - c)**2 / (2 * sigma**2)) for c in centers]
  return res

def RBF_layer_output(x, centers, sigma):
  # res = []
  # for x_ in x: 
  #   res.append(RBF_kernel(x_, centers, sigma))
  # res = [RBF_kernel(x_, centers, sigma) for x_ in x]
  # res = [0 for x_ in x]
  res = [RBF_kernel(x_, centers, sigma) for x_ in x]

  return res

class RBF(keras.layers.Layer):

  def __init__(self, num_units):
    self.num_units = num_units
    super(RBF, self).__init__()

  def build(self, input_shape):
    # shape1 = tf.TensorShape((input_shape[1], self.num_units))
    # shape2 = tf.TensorShape((1, self.num_units))
    self.centers = self.add_weight(shape=(input_shape[1], self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    self.sigma = self.add_weight(shape=(1, self.num_units), initializer=tf.initializers.RandomNormal, trainable=False) # TODO use custom initializer
    # self.betas = np.ones(self.num_units) / (2 * (sigma**2))
    # self.sigma    = self.add_weight(shape=1, initializer=tf.initializers.RandomNormal, trainable=False)
    super(RBF, self).build(input_shape)

  def call(self, inputs):
    # C = tf.expand_dims(self.centers, -1)
    # D = inputs - C

    diff = []
    for x in inputs:
      diff.append(x - tf.transpose(self.centers))

    # print(tf.convert_to_tensor(diff))

    D = inputs - tf.transpose(self.centers)
    print(D)
    return tf.exp(-tf.norm(D, axis=1)**2) / (2 * self.sigma[0]**2) # TODO change sigma[0]

  def get_config(self):
    base_config = super(RBF, self).get.config
    base_config['num_units'] = self.num_units
    return base_config

  def compute_params(self, inputs):
    kmeans = KMeans(n_clusters=self.num_units, random_state=0, copy_x=True).fit(inputs)
    self.centers = kmeans.cluster_centers_
    d_max = ComputeMaxDistance(self.centers, self.centers)
    self.sigma = d_max / (math.sqrt(2 * self.centers.shape[0]))



In [343]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import preprocessing
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import utils
import rbf_layer

# ---------------------------------------------------------------------------- #
#                                   read data                                  #
# ---------------------------------------------------------------------------- #

# data_url = "http://lib.stat.cmu.edu/datasets/boston"
data = "boston.csv"
raw_df = pd.read_csv(data, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

# ---------------------------------------------------------------------------- #
#                                normalize data                                #
# ---------------------------------------------------------------------------- #

preprocessing.scale(X, copy=False)
preprocessing.scale(y, copy=False)

# ---------------------------------------------------------------------------- #
#                      split data to training and testing                      #
# ---------------------------------------------------------------------------- #

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=0)

n_train = X_train.shape[0]
hidden_size = int(0.1 * n_train)

In [350]:
rbf_layer_ = RBF(hidden_size)
rbf_layer_.compute_params(X_train)
# rbf_layer_.compute_params(X_train)

from keras import layers

model = keras.Sequential()
model.add(Input(shape=(13,)))
# model.add(rbf_layer_)
# model.add(layers.Dense(12, input_shape=(13,), activation='relu'))
model.add(rbf_layer_)
# model.add(layers.Dense(8, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


Tensor("rbf_77/sub:0", shape=(37, 13), dtype=float32)


In [351]:
# model.compile(optimizer=tf.keras.optimizers.Adam(),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
#               metrics=['accuracy'])

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile()
# model.fit(X_train, y_train, epochs=5, batch_size=10)

# train model
# history = model.fit(X_train, y_train)


In [353]:
# print(history.params)
# model.summary()
model.predict(X_train[0])
# a = tf.constant([1, 2])
# print(type(a))

Tensor("sequential_77/rbf_77/sub:0", shape=(37, 13), dtype=float32)


InvalidArgumentError:  Incompatible shapes: [3,13] vs. [37,13]
	 [[node sequential_77/rbf_77/sub
 (defined at C:\Users\gitop\AppData\Local\Temp\ipykernel_13384\2948013630.py:66)
]] [Op:__inference_predict_function_24187]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_77/rbf_77/sub:
In[0] IteratorGetNext (defined at c:\Python39\lib\site-packages\keras\engine\training.py:1610)	
In[1] sequential_77/rbf_77/transpose:

Operation defined at: (most recent call last)
>>>   File "c:\Python39\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "c:\Python39\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "C:\Users\gitop\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "c:\Python39\lib\site-packages\traitlets\config\application.py", line 976, in launc

In [332]:

# x = tf.constant([1, 2, 3])

x = tf.constant([[1, 2, 3], [4, 5, 6]])
centers = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

res = []

for x_ in x:
    res.append(x_-centers)

# print(res)

# result = [res_ for res_ in res]
# print(res.shape)

# for res_ in res:
#     print(res_)

# tf.concat([res[0], res[1]], 0)
# rrrr = tf.expand_dims(res, 0)
rrrr = tf.convert_to_tensor(res)
print(rrrr)

tf.Tensor(
[[[ 0  0  0]
  [-3 -3 -3]
  [-6 -6 -6]]

 [[ 3  3  3]
  [ 0  0  0]
  [-3 -3 -3]]], shape=(2, 3, 3), dtype=int32)
